In [ ]:
import os, glob, json, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as ply
import plotly
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

In [ ]:
userhome = os.path.expanduser('~')

# Create directory list

In [ ]:
dir_list = glob.glob(userhome + r'/different-diff/dataset/hbase/*/**')
print ("Found " + str(len(dir_list)) + " dirs")

In [ ]:
with open ("dir_list.txt", mode="wt", encoding="utf-8") as myfile:
    myfile.write('\n'.join(dir_list))
print ("File dir_list.txt has been created")

# Load directory list from file 

In [ ]:
txt_file = open("dir_list.txt", "r")
dir_list = txt_file.read().split('\n')
print ("Found " + str(len(dir_list)) + " dirs")

# Defining function to extract commit log

In [ ]:
def execute_shell_command(cmd, work_dir):
    """Executes a shell command in a subprocess, waiting until it has completed.
 
    :param cmd: Command to execute.
    :param work_dir: Working directory path.
    """
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

Defining Repository

In [ ]:
repository  = os.path.dirname(userhome + r'/different-diff/dataset/hbase/') 

List of git command

In [ ]:
cmd = ['git log -w --unified=5 | wc -l', 'git log --unified=5 | wc -l', 
       'git log -U0 -w | wc -l', 'git log -U0 | wc -l', 
       'git log -w --patch | wc -l', 'git log --patch | wc -l', 
       'git log -U0 -w --ignore-blank-lines | wc -l']

In [ ]:
result = []
for i in range(0,len(cmd)):
    sys.stdout.write("\rApplying git command = %i" % (i+1))
    sys.stdout.flush()
    result.append(int(str(execute_shell_command(cmd[i], repository)).replace("b",'').replace("'",'').
                      replace(' ','').replace("\\n,",'').replace("(",'').replace(")",'')))

print ("\n")
x = len(result)
y = x
for item in result:
    print (cmd[x-y] , "\t\t\t = " , item)
    y = y-1

In [ ]:
trace = go.Bar(
    x=cmd,
    y=result,
    name='number of line'
)

data = [trace]
layout = go.Layout(
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)

# Extracting Commit ID

In [ ]:
command = "git log --pretty='%H'"

In [ ]:
com_ID = str(execute_shell_command(command, repository)).replace("b'",'').replace("'",'').replace("(",'').replace(")",'').replace(",",'').split('\\n')
for i, cid in enumerate(com_ID):
    if cid == ' ':
        del com_ID[i]
com_ID

In [ ]:
with open ("commit_id.txt", mode="wt", encoding="utf-8") as myfile:
    myfile.write('\n'.join(com_ID))
print ("File commit_id.txt has been created")

In [ ]:
com_ID[len(com_ID)-1]

In [ ]:
#set directory to save metrics
metrics_dir = userhome + r'/different-diff/analyze/analyze_hbase/metrics/'

# Applying git diff commitID_1 commitID2 --shortstat

In [ ]:
diff = []
for i in range(0, len(com_ID)-1):
    sys.stdout.write('\rExtracting diff = %i' % (i+1) + ' out of %i' % (len(com_ID)-1))
    sys.stdout.flush()
    diff_cmd = "git diff " + com_ID[i+1] + " " + com_ID[i] + " --shortstat"
    diff.append(str(execute_shell_command(diff_cmd, repository)).replace("b'",'').replace("'",'').replace("\\n",' ').replace("( ",'').replace(" )",''))

print ("\n")
for item in diff:   
    print (item)

In [ ]:
for x in range(0,len(diff)):
    line = diff[x]
    if "deletion" not in diff[x]:
        index = line.find('(+)')
        diff[x] = line[:index] + ', 0 deletions(-) ,'
    if "insertion" not in diff[x]:
        index = diff[x].find(',')
        diff[x] = line[:index] + ', 0 insertions(+)' + line[index:]
    else:
        diff[x] = diff[x]

In [ ]:
diff_metrics = []
for j in range(0,len(diff)):
    diff_metrics.append(re.findall(r'\d+', diff[j]))

for k in range(0, len(diff_metrics)):
    diff_metrics[k] = list(map(int, diff_metrics[k]))

In [ ]:
with open(metrics_dir + 'metrics1_git_diff_cid1_cid2_shortstat.csv', 'w') as csvfile:
    header = ['files changed', 'insertions(+)', 'deletions(-)']
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)
    for row in diff_metrics:
        writer.writerow(row)

In [ ]:
data1 = pd.read_csv(metrics_dir + 'metrics1_git_diff_cid1_cid2_shortstat.csv')
file_sum1 = data1['files changed'].sum()
ins_sum1 = data1['insertions(+)'].sum()
del_sum1 = data1['deletions(-)'].sum()
print ("Sum of files changed = %i" % file_sum1)
print ("Sum of insertions(+) = %i" % ins_sum1)
print ("Sum of deletions(-)  = %i" % del_sum1)
saved_column1 = data1[header]
saved_column1.head()

In [ ]:
mygraph = data1.sum().plot(figsize=(15,5), kind='bar', title = 'Number of changed line of code')
mygraph.set_xlabel("Type of change")
mygraph.set_ylabel("Quantity")

# Applying git diff -w --ignore-blank-lines commitID_1 commitID2 --shortstat

In [ ]:
diff = []
for i in range(0, len(com_ID)-1):
    sys.stdout.write('\rExtracting diff = %i' % (i+1) + ' out of %i' % (len(com_ID)-1))
    sys.stdout.flush()
    diff_cmd = "git diff -w --ignore-blank-lines " + com_ID[i+1] + " " + com_ID[i] + " --shortstat"
    diff.append(str(execute_shell_command(diff_cmd, repository)).replace("b'",'').replace("'",'').replace("\\n",' ').replace("( ",'').replace(" )",''))

print ("\n")
for item in diff:   
    print (item)

In [ ]:
for x in range(0,len(diff)):
    line = diff[x]
    if "deletion" not in diff[x]:
        index = line.find('(+)')
        diff[x] = line[:index] + ', 0 deletions(-) ,'
    if "insertion" not in diff[x]:
        index = diff[x].find(',')
        diff[x] = line[:index] + ', 0 insertions(+)' + line[index:]
    else:
        diff[x] = diff[x]

In [ ]:
diff_metrics = []
for j in range(0,len(diff)):
    diff_metrics.append(re.findall(r'\d+', diff[j]))

for k in range(0, len(diff_metrics)):
    diff_metrics[k] = list(map(int, diff_metrics[k]))

In [ ]:
with open(metrics_dir + 'metrics2_git_diff_w_ignoreblanklines_cid1_cid2_shortstat.csv', 'w') as csvfile:
    header = ['files changed', 'insertions(+)', 'deletions(-)']
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)
    for row in diff_metrics:
        writer.writerow(row)

In [ ]:
data2 = pd.read_csv(metrics_dir + 'metrics2_git_diff_w_ignoreblanklines_cid1_cid2_shortstat.csv')
file_sum2 = data2['files changed'].sum()
ins_sum2 = data2['insertions(+)'].sum()
del_sum2 = data2['deletions(-)'].sum()
print ("Sum of files changed = %i" % file_sum2)
print ("Sum of insertions(+) = %i" % ins_sum2)
print ("Sum of deletions(-)  = %i" % del_sum2)
saved_column2 = data2[header]
saved_column2.head()

In [ ]:
mygraph = data2.sum().plot(figsize=(15,5), kind='bar', title = 'Number of changed line of code')
mygraph.set_xlabel("Type of change")
mygraph.set_ylabel("Quantity")

In [ ]:
data = [data1, data2]
table_sum = []
for i in range(0, len(data)):
    summ = 'command %i' % (i+1), data[i]['files changed'].sum(), data[i]['insertions(+)'].sum(), data[i]['deletions(-)'].sum()
    table_sum.append(summ)

for item in table_sum:
    print (item)

In [ ]:
df = pd.DataFrame(table_sum, columns = ['command', 'files changed', 'insertions', 'deletions'])
df.to_csv(metrics_dir + 'metrics_sum.csv')

In [ ]:
trace1 = go.Bar(
    x=['files changed', 'insertions', 'deletions'],
    y=[file_sum1, ins_sum1, del_sum1],
    name='git diff commit1 commit2'
)
trace2 = go.Bar(
    x=['files changed', 'insertions', 'deletions'],
    y=[file_sum2, ins_sum2, del_sum2],
    name='git diff -w --ignore-blank-lines commit1 commit2'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)

# git diff -w --ignore-blank-lines --diff-algorithm=myers commit1 commit2 --shortstat

In [ ]:
diff = []
for i in range(0, len(com_ID)-1):
    sys.stdout.write('\rExtracting diff = %i' % (i+1) + ' out of %i' % (len(com_ID)-1))
    sys.stdout.flush()
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=myers " + com_ID[i+1] + " " + com_ID[i] + " --shortstat"
    diff.append(str(execute_shell_command(diff_cmd, repository)).replace("b'",'').replace("'",'').replace("\\n",' ').replace("( ",'').replace(" )",''))

print ("\n")
for item in diff:   
    print (item)

In [ ]:
for x in range(0,len(diff)):
    line = diff[x]
    if "deletion" not in diff[x]:
        index = line.find('(+)')
        diff[x] = line[:index] + ', 0 deletions(-) ,'
    if "insertion" not in diff[x]:
        index = diff[x].find(',')
        diff[x] = line[:index] + ', 0 insertions(+)' + line[index:]
    else:
        diff[x] = diff[x]

In [ ]:
diff_metrics = []
for j in range(0,len(diff)):
    diff_metrics.append(re.findall(r'\d+', diff[j]))

for k in range(0, len(diff_metrics)):
    diff_metrics[k] = list(map(int, diff_metrics[k]))

In [ ]:
with open(metrics_dir + 'metrics3_git_diff_w_ignoreblanklines_myers_cid1_cid2_shortstat.csv', 'w') as csvfile:
    header = ['files changed', 'insertions(+)', 'deletions(-)']
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)
    for row in diff_metrics:
        writer.writerow(row)

In [ ]:
data3 = pd.read_csv(metrics_dir + 'metrics3_git_diff_w_ignoreblanklines_myers_cid1_cid2_shortstat.csv')
file_sum3 = data3['files changed'].sum()
ins_sum3 = data3['insertions(+)'].sum()
del_sum3 = data3['deletions(-)'].sum()
print ("Sum of files changed = %i" % file_sum3)
print ("Sum of insertions(+) = %i" % ins_sum3)
print ("Sum of deletions(-)  = %i" % del_sum3)
saved_column3 = data3[header]
saved_column3.head()

# git diff -w --ignore-blank-lines --diff-algorithm=histogram commit1 commit2 --shortstat

In [ ]:
diff = []
for i in range(0, len(com_ID)-1):
    sys.stdout.write('\rExtracting diff = %i' % (i+1) + ' out of %i' % (len(com_ID)-1))
    sys.stdout.flush()
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=histogram " + com_ID[i+1] + " " + com_ID[i] + " --shortstat"
    diff.append(str(execute_shell_command(diff_cmd, repository)).replace("b'",'').replace("'",'').replace("\\n",' ').replace("( ",'').replace(" )",''))

print ("\n")
for item in diff:   
    print (item)

In [ ]:
for x in range(0,len(diff)):
    line = diff[x]
    if "deletion" not in diff[x]:
        index = line.find('(+)')
        diff[x] = line[:index] + ', 0 deletions(-) ,'
    if "insertion" not in diff[x]:
        index = diff[x].find(',')
        diff[x] = line[:index] + ', 0 insertions(+)' + line[index:]
    else:
        diff[x] = diff[x]

In [ ]:
diff_metrics = []
for j in range(0,len(diff)):
    diff_metrics.append(re.findall(r'\d+', diff[j]))

for k in range(0, len(diff_metrics)):
    diff_metrics[k] = list(map(int, diff_metrics[k]))

In [ ]:
with open(metrics_dir + 'metrics4_git_diff_w_ignoreblanklines_histogram_cid1_cid2_shortstat.csv', 'w') as csvfile:
    header = ['files changed', 'insertions(+)', 'deletions(-)']
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)
    for row in diff_metrics:
        writer.writerow(row)

In [ ]:
data4 = pd.read_csv(metrics_dir + 'metrics4_git_diff_w_ignoreblanklines_histogram_cid1_cid2_shortstat.csv')
file_sum4 = data4['files changed'].sum()
ins_sum4 = data4['insertions(+)'].sum()
del_sum4 = data4['deletions(-)'].sum()
print ("Sum of files changed = %i" % file_sum4)
print ("Sum of insertions(+) = %i" % ins_sum4)
print ("Sum of deletions(-)  = %i" % del_sum4)
saved_column4 = data4[header]
saved_column4.head()

In [ ]:
trace1 = go.Bar(
    x=['files changed', 'insertions', 'deletions'],
    y=[file_sum1, ins_sum1, del_sum1],
    name='git diff'
)
trace2 = go.Bar(
    x=['files changed', 'insertions', 'deletions'],
    y=[file_sum2, ins_sum2, del_sum2],
    name='git diff -w'
)
trace3 = go.Bar(
    x=['files changed', 'insertions', 'deletions'],
    y=[file_sum3, ins_sum3, del_sum3],
    name='git diff -w --diff-algorithm=myers'
)
trace4 = go.Bar(
    x=['files changed', 'insertions', 'deletions'],
    y=[file_sum4, ins_sum4, del_sum4],
    name='git diff -w --diff-algorithm=histogram'
)

data = [trace1, trace2, trace3, trace4]
layout = go.Layout(
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)

# git diff -w --ignore-blank-lines --diff-algorithm=minimal commit1 commit2 --shortstat

In [ ]:
diff = []
for i in range(0, len(com_ID)-1):
    sys.stdout.write('\rExtracting diff = %i' % (i+1) + ' out of %i' % (len(com_ID)-1))
    sys.stdout.flush()
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=minimal " + com_ID[i+1] + " " + com_ID[i] + " --shortstat"
    diff.append(str(execute_shell_command(diff_cmd, repository)).replace("b'",'').replace("'",'').replace("\\n",' ').replace("( ",'').replace(" )",''))

print ("\n")
for item in diff:   
    print (item)

In [ ]:
for x in range(0,len(diff)):
    line = diff[x]
    if "deletion" not in diff[x]:
        index = line.find('(+)')
        diff[x] = line[:index] + ', 0 deletions(-) ,'
    if "insertion" not in diff[x]:
        index = diff[x].find(',')
        diff[x] = line[:index] + ', 0 insertions(+)' + line[index:]
    else:
        diff[x] = diff[x]

In [ ]:
diff_metrics = []
for j in range(0,len(diff)):
    diff_metrics.append(re.findall(r'\d+', diff[j]))

for k in range(0, len(diff_metrics)):
    diff_metrics[k] = list(map(int, diff_metrics[k]))

In [ ]:
with open(metrics_dir + 'metrics5_git_diff_w_ignoreblanklines_minimal_cid1_cid2_shortstat.csv', 'w') as csvfile:
    header = ['files changed', 'insertions(+)', 'deletions(-)']
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)
    for row in diff_metrics:
        writer.writerow(row)

In [ ]:
data5 = pd.read_csv(metrics_dir + 'metrics5_git_diff_w_ignoreblanklines_minimal_cid1_cid2_shortstat.csv')
file_sum5 = data5['files changed'].sum()
ins_sum5 = data5['insertions(+)'].sum()
del_sum5 = data5['deletions(-)'].sum()
print ("Sum of files changed = %i" % file_sum5)
print ("Sum of insertions(+) = %i" % ins_sum5)
print ("Sum of deletions(-)  = %i" % del_sum5)
saved_column5 = data5[header]
saved_column5.head()

# git diff -w --ignore-blank-lines --diff-algorithm=patience commit1 commit2 --shortstat

In [ ]:
diff = []
for i in range(0, len(com_ID)-1):
    sys.stdout.write('\rExtracting diff = %i' % (i+1) + ' out of %i' % (len(com_ID)-1))
    sys.stdout.flush()
    diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=patience " + com_ID[i+1] + " " + com_ID[i] + " --shortstat"
    diff.append(str(execute_shell_command(diff_cmd, repository)).replace("b'",'').replace("'",'').replace("\\n",' ').replace("( ",'').replace(" )",''))

print ("\n")
for item in diff:   
    print (item)

In [ ]:
for x in range(0,len(diff)):
    line = diff[x]
    if "deletion" not in diff[x]:
        index = line.find('(+)')
        diff[x] = line[:index] + ', 0 deletions(-) ,'
    if "insertion" not in diff[x]:
        index = diff[x].find(',')
        diff[x] = line[:index] + ', 0 insertions(+)' + line[index:]
    else:
        diff[x] = diff[x]

In [ ]:
diff_metrics = []
for j in range(0,len(diff)):
    diff_metrics.append(re.findall(r'\d+', diff[j]))

for k in range(0, len(diff_metrics)):
    diff_metrics[k] = list(map(int, diff_metrics[k]))

In [ ]:
with open(metrics_dir + 'metrics6_git_diff_w_ignoreblanklines_patience_cid1_cid2_shortstat.csv', 'w') as csvfile:
    header = ['files changed', 'insertions(+)', 'deletions(-)']
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(header)
    for row in diff_metrics:
        writer.writerow(row)

In [ ]:
data6 = pd.read_csv(metrics_dir + 'metrics6_git_diff_w_ignoreblanklines_patience_cid1_cid2_shortstat.csv')
file_sum6 = data6['files changed'].sum()
ins_sum6 = data6['insertions(+)'].sum()
del_sum6 = data6['deletions(-)'].sum()
print ("Sum of files changed = %i" % file_sum6)
print ("Sum of insertions(+) = %i" % ins_sum6)
print ("Sum of deletions(-)  = %i" % del_sum6)
saved_column6 = data6[header]
saved_column6.head()

In [ ]:
trace3 = go.Bar(
    x=['insertions', 'deletions'],
    y=[ins_sum3, del_sum3],
    text=[ins_sum3, del_sum3],
    textposition='auto',
    name='--diff-algorithm=myers'
)
trace4 = go.Bar(
    x=['insertions', 'deletions'],
    y=[ins_sum4, del_sum4],
    text=[ins_sum4, del_sum4],
    textposition='auto',
    name='--diff-algorithm=histogram'
)
trace5 = go.Bar(
    x=['insertions', 'deletions'],
    y=[ins_sum5, del_sum5],
    text=[ins_sum5, del_sum5],
    textposition='auto',
    name='--diff-algorithm=minimal'
)
trace6 = go.Bar(
    x=['insertions', 'deletions'],
    y=[ins_sum6, del_sum6],
    text=[ins_sum6, del_sum6],
    textposition='auto',
    name='--diff-algorithm=patience'
)
data = [trace3, trace4, trace5, trace6]
layout = go.Layout(
    title = 'Comparison of the number of changes between algorithms in HBase Project',
    barmode='group'
)

init_notebook_mode(connected=True)
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)